In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import re
from sklearn.pipeline import Pipeline
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
import pickle
import tweepy
from IPython.display import clear_output

In [21]:
users = pd.read_csv('data/full/train.csv')

In [22]:
users = users[['user_id','target']]

In [23]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/matteo/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  int

In [24]:
stemmer = SnowballStemmer("english", ignore_stopwords=True)

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc:(stemmer.stem(w) for w in analyzer(doc))

In [25]:
tweets = tweets.drop_duplicates()

In [26]:
tweets = pd.merge(tweets, users, on='user_id')

In [27]:
def remove_rt(x):
    if 'RT' in x:
        x = x.replace('RT', '')
        try:
            return x[x.rindex(':')+2:]
        except:
            return x
    else:
        return x

In [28]:
stop_words = stopwords.words('english')

def remove_stop(x):
    return [word for word in x.split() if word not in stop_words]

In [29]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_rt(x))
tweets['full_text'] = tweets['full_text'].apply(lambda x: re.sub(r'^\/t.co\/[^\s]+', '', x))
tweets['full_text'] = tweets['full_text'].apply(lambda x: re.sub(r'[^\s]+\/t.co\/[^\s]+', '', x))
tweets['full_text'] = tweets['full_text'].apply(lambda x: re.sub(r'[^\w\s]','',x))
tweets['full_text'] = tweets['full_text'].apply(lambda x: x.lower())
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_stop(x))
tweets['full_text'] = tweets['full_text'].astype(str)
tweets['full_text'] = tweets['full_text'][tweets.full_text!='[]']

In [30]:
nb = pickle.load( open( "../scripts/nb.model", "rb" ) )

In [31]:
proba = pd.DataFrame(columns=['nb_0', 'nb_1', 'nb_2', 'nb_3', 'nb_4', 'target']) 

In [32]:
predictions = []
i = 0

for usr in users.itertuples():
    
    i+=1
    clear_output()
    print(i)
    tweets_list = tweets[tweets.user_id == usr.user_id]['full_text']
    tweets_list = tweets_list[tweets_list.notnull()]
    if len(tweets_list[tweets_list.notnull()]) != 0:
        pred = nb.predict_proba(tweets_list)
        predictions.append(np.append(np.mean(pred, axis=0), usr.target))
    else:
        predictions.append([0.2,0.2,0.2,0.2,0.2,usr.target])

25106


In [33]:
prob = pd.DataFrame(predictions, columns=['nb_0', 'nb_1', 'nb_2', 'nb_3', 'nb_4', 'target'])

In [36]:
prob['target'] = prob.target.astype(int)

In [40]:
prob.head()

,nb_0,nb_1,nb_2,nb_3,nb_4,target
0,0.286265,0.191064,0.301550,0.038845,0.182275,0
1,0.362127,0.127418,0.358035,0.041214,0.111206,0
2,0.509541,0.211704,0.092972,0.043473,0.142310,0
3,0.466294,0.079028,0.164804,0.126437,0.163437,0
4,0.400717,0.168731,0.214842,0.045414,0.170296,0


In [41]:
prob.to_csv('data/full/predictions1.csv')